In [91]:
import requests
import pandas as pd
from time import sleep

import kagglehub
from kagglehub import KaggleDatasetAdapter

import re

In [92]:
file_path = "movies.csv"

# Load the latest version
movies = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "akshaypawar7/millions-of-movies",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", movies.head())

C:\Users\Владислав\AppData\Local\Temp\ipykernel_2780\183436944.py:4: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  movies = kagglehub.load_dataset(


First 5 records:        id                             title                            genres  \
0  615656                 Meg 2: The Trench     Action-Science Fiction-Horror   
1  758323               The Pope's Exorcist           Horror-Mystery-Thriller   
2  533535              Deadpool & Wolverine     Action-Comedy-Science Fiction   
3  667538  Transformers: Rise of the Beasts  Action-Adventure-Science Fiction   
4  693134                    Dune: Part Two         Science Fiction-Adventure   

  original_language  \
0                en   
1                en   
2                en   
3                en   
4                en   

                                                                                                                                                                                                                                                                                                                                      overview  \
0                  

In [93]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722317 entries, 0 to 722316
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    722317 non-null  int64  
 1   title                 722311 non-null  object 
 2   genres                512000 non-null  object 
 3   original_language     722317 non-null  object 
 4   overview              604074 non-null  object 
 5   popularity            722317 non-null  float64
 6   production_companies  337391 non-null  object 
 7   release_date          670768 non-null  object 
 8   budget                722317 non-null  float64
 9   revenue               722317 non-null  float64
 10  runtime               687982 non-null  float64
 11  status                722317 non-null  object 
 12  tagline               108476 non-null  object 
 13  vote_average          722317 non-null  float64
 14  vote_count            722317 non-null  float64
 15  

In [94]:
movies.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival.,8763.998,Apelles Entertainment-Warner Bros. Pictures-di Bonaventura Pictures-CMC Pictures-Gravity Pictures-China Film Group Corporation,2023-08-02,129000000.0,3.520565e+08,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio Peris-Mencheta-Skyler Samuels-Cliff Curtis-Page Kennedy-Sienna Guillory-Melissanthi Mahut-Kiran Sonia Sawar-Felix Mayr-Whoopie van Raam-Guo Tao-Robin Hill-Dai Lele-Sui Fong Ivy Tsui-Stewart Alexander-Li Xin-Billy Clements-Ron Smoorenburg-Rui Shang-Sara Dee-Jonny James-Bai Narisu-Kenneth Won-Able Wanamakok,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-616747-1083862-614930-1003581-980489-614479-1040148-872585-57084-667538-1149381-457332-724209-884605
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden.,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Worldwide Katz-Loyola Productions-FFILME.RO,2023-04-05,18000000.0,6.567582e+07,103.0,Released,"Inspired by the actual files of Father Gabriele Amorth, Chief Exorcist of the Vatican.",7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco Nero-Peter DeSouza-Feighoney-Laurel Marsden-Cornell John-Ryan O'Grady-Bianca Bardoe-Santi Bayón-Paloma Bloyd-Alessandro Gruttadauria-River Hawkins-Jordi Collet-Carrie Munro-Marc Velasco-Edward Harper-Jones-Matthew Sim-Victor Solé-Tom Bonington-Andrea Dugoni-Ed White-Laila Barwick-Gennaro Diana-Pablo Raybould-Ralph Ineson-Derek Carroll-Ella Cannon,spain-rome italy-vatican-pope-pig-possession-conspiracy-devil-exorcist-skepticism-catholic priest-1980s-supernatural horror,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-916224-1023313-1033219-980078-842945-943822-816904-804150-638974-649609-603692-849869-809787-776835-1104040
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary Deadpool behind him. But when his homeworld faces an existential threat Wade must reluctantly suit-up again with an even more reluctant Wolverine.,5410.496,Marvel Studios-Maximum Effort-21 Laps Entertainment-20th Century Studios-Kevin Feige Productions-TSG Entertainment,2024-07-24,200000000.0,1.326387e+09,128.0,Released,Come together.,7.765,3749.0,Ryan Reynolds-Hugh Jackman-Emma Corrin-Matthew Macfadyen-Dafne Keen-Jon Favreau-Morena Baccarin-Rob Delaney-Leslie Uggams-Jennifer Garner-Wesley Snipes-Channing Tatum-Chris Evans-Henry Cavill-Wunmi Mosaku-Aaron Stanford-Tyler Mane-Karan Soni-Brianna Hildebrand-Shioli Kutsuna-Stefan Kapicic-Randal Reeder-Lewis Tan-Nick Pauley-Sonita Henry-Ryan McKen-Nanak Phlora-Aydin Ahmed-Leemore Marrett Jr.-James Dryden-Ollie Palmer-Greg Hemphill-Aaron W Reed-Mike Waters-Rob McElhenney-James Reynolds-Ed Kear-Paul G. Raymond-Blake Lively-Inez Reynolds-Nathan Fillion-Matthew McConaughey-OIin Reynolds-Paul Mullin-Alex Kyshkovych-Chris Hemsworth-Billy Clements-Daniel Medina Ramos-Jade Lye-Nilly Cetin-Eduardo Gago Muñoz-Chloe Kibble-Curtis Rowland Small-Ayesha Hussain-Jessica Walker-Harry Holland-Kevin Fortin,hero-superhero-anti hero-mutant-breaking the fourth wall-marvel cinematic universe (mcu)-mutants-superhero teamup,/8cdWjvZQUExUUTzyp4t6ED

In [95]:
movies_not_missing = movies[~(movies['title'].isna()) &
                ~(movies['genres'].isna()) &
                ~(movies['overview'].isna()) &
                ~(movies['poster_path'].isna())] 

In [96]:
good_movies_not_missing = movies_not_missing[movies_not_missing['vote_average'] >= 6]

In [97]:
good_movies = good_movies_not_missing.sort_values(by='popularity', ascending=False)[:1500]

In [98]:
good_movies['id_description'] = good_movies[['id', 'overview']].astype(str).agg(" ".join, axis=1)

In [99]:
def preprocess_text(row):
    description = str(row["overview"])
    keywords_raw = str(row["keywords"])
    genres_raw = str(row.get("genres", ""))
    
    keywords_cleaned = re.sub(r'[-]', ' ', keywords_raw)
    keywords_cleaned = re.sub(r'\s+', ' ', keywords_cleaned)
    genres_cleaned = re.sub(r'[-]', ' ', genres_raw)
    
    full_text = f'{row["id"]} {description.strip()} {keywords_cleaned.strip()} {genres_cleaned.strip()}'
    return full_text

In [100]:
good_movies["id_description_keywords_genres"] = good_movies.apply(preprocess_text, axis=1)

In [101]:
good_movies

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations,id_description,id_description_keywords_genres
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival.,8763.998,Apelles Entertainment-Warner Bros. Pictures-di Bonaventura Pictures-CMC Pictures-Gravity Pictures-China Film Group Corporation,2023-08-02,129000000.0,3.520565e+08,...,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio Peris-Mencheta-Skyler Samuels-Cliff Curtis-Page Kennedy-Sienna Guillory-Melissanthi Mahut-Kiran Sonia Sawar-Felix Mayr-Whoopie van Raam-Guo Tao-Robin Hill-Dai Lele-Sui Fong Ivy Tsui-Stewart Alexander-Li Xin-Billy Clements-Ron Smoorenburg-Rui Shang-Sara Dee-Jonny James-Bai Narisu-Kenneth Won-Able Wanamakok,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-616747-1083862-614930-1003581-980489-614479-1040148-872585-57084-667538-1149381-457332-724209-884605,615656 An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival.,615656 An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival. based on novel or book sequel kaiju Action Science Fiction Horror
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden.,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Worldwide Katz-Loyola Productions-FFILME.RO,2023-04-05,18000000.0,6.567582e+07,...,"Inspired by the actual files of Father Gabriele Amorth, Chief Exorcist of the Vatican.",7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco Nero-Peter DeSouza-Feighoney-Laurel Marsden-Cornell John-Ryan O'Grady-Bianca Bardoe-Santi Bayón-Paloma Bloyd-Alessandro Gruttadauria-River Hawkins-Jordi Collet-Carrie Munro-Marc Velasco-Edward Harper-Jones-Matthew Sim-Victor Solé-Tom Bonington-Andrea Dugoni-Ed White-Laila Barwick-Gennaro Diana-Pablo Raybould-Ralph Ineson-Derek Carroll-Ella Cannon,spain-rome italy-vatican-pope-pig-possession-conspiracy-devil-exorcist-skepticism-catholic priest-1980s-supernatural horror,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-916224-1023313-1033219-980078-842945-943822-816904-804150-638974-649609-603692-849869-809787-776835-1104040,758323 Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden.,758323 Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden. spain rome italy vatican pope pig possession conspiracy devil exorcist skepticism catholic priest 1980s supernatural horror Horror Mystery Thriller
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary Deadpool behind him. But when his homeworld faces an existential threat Wade must reluctantly suit-up again with an even more reluctant Wolverine.,5410.496,Marvel Studios-Maximum Effort-21 Laps E

In [102]:
good_movies['poster_url'] = good_movies['poster_path'].apply(
    lambda x: f"https://image.tmdb.org/t/p/w500{x}" if pd.notna(x) else None
)

In [103]:
ready_movies = good_movies[['id', 'title', 'genres', 'original_language', 'popularity', 'release_date', 'runtime', 'tagline', 'vote_average', 'poster_url', 'overview', 'id_description', 'id_description_keywords_genres']]

In [104]:
ready_movies

,id,title,genres,original_language,popularity,release_date,runtime,tagline,vote_average,poster_url,overview,id_description,id_description_keywords_genres
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,8763.998,2023-08-02,116.0,Back for seconds.,7.079,https://image.tmdb.org/t/p/w500/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival.,615656 An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival.,615656 An exploratory dive into the deepest depths of the ocean of a daring research team spirals into chaos when a malevolent mining operation threatens their mission and forces them into a high-stakes battle for survival. based on novel or book sequel kaiju Action Science Fiction Horror
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,5953.227,2023-04-05,103.0,"Inspired by the actual files of Father Gabriele Amorth, Chief Exorcist of the Vatican.",7.433,https://image.tmdb.org/t/p/w500/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden.,758323 Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden.,758323 Father Gabriele Amorth Chief Exorcist of the Vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the Vatican has desperately tried to keep hidden. spain rome italy vatican pope pig possession conspiracy devil exorcist skepticism catholic priest 1980s supernatural horror Horror Mystery Thriller
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,5410.496,2024-07-24,128.0,Come together.,7.765,https://image.tmdb.org/t/p/w500/8cdWjvZQUExUUTzyp4t6EDMubfO.jpg,A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary Deadpool behind him. But when his homeworld faces an existential threat Wade must reluctantly suit-up again with an even more reluctant Wolverine.,533535 A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary Deadpool behind him. But when his homeworld faces an existential threat Wade must reluctantly suit-up again with an even more reluctant Wolverine.,533535 A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary Deadpool behind him. But when his homeworld faces an existential threat Wade must reluctantly suit-up again with an even more reluctant Wolverine. hero superhero anti hero mutant breaking the fourth wall marvel cinematic universe (mcu) mutants superhero teamup Action Comedy Science Fiction
3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,5409.104,2023-06-06,127.0,Unite or fall.,7.340,https://image.tmdb.org/t/p/w500/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,When a new threat capable of destroying the entire planet emerges Optimus Prime and the Autobots must team up with a powerful faction known as the Maximals. With the fate of humanity hanging in the balance humans Noah and Elena will do whatever it takes to help the Transformers as they engage in the ultimate battle to save Earth.,667538 When a new threat capable of destroying the entire planet emerges Optimus Prime and the Autobots must team up with a powerful faction known as the Maximals. With the fate of humanity hanging in the balance humans Noah and Elena will do whatever it takes to help the Transformers as they engage in the ultimate battle to save Earth.,66

In [105]:
ready_movies.to_csv("movies_cleaned.csv", index = False)